# SLR-Aligned Violence Detection System
## Academic Implementation

---

## Overview

This notebook implements a  violence detection system aligned with Systematic Literature Review (SLR) methodology.

---

## System Architecture

### Core Components
- **YOLOv8 Pose Estimation** with multi-person tracking
- **Biomechanical Feature Extraction** (angles, deltas, spatial metrics)
- **BiLSTM with Attention Mechanism** for temporal modeling
- **BoTSORT-inspired Tracking** for multi-person scenarios

---

## Performance Target

| Metric    | Target  |
|-----------|---------|
| Accuracy  | 80-90%  |
| Dataset   | RWF-2000|

---

## Expected Outcomes

- Violence detection in multi-person scenarios
- Interpretable biomechanical feature analysis
- Temporal pattern recognition via attention mechanisms
- Reproducibility

---


# Section 1: Environment Setup & Imports
## System Configuration and Dependency Management


In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers, regularizers

# Scikit-learn utilities
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import (classification_report, confusion_matrix, roc_curve, 
                             precision_recall_curve, roc_auc_score, 
                             average_precision_score, f1_score, matthews_corrcoef,
                             precision_score, recall_score, accuracy_score)
from sklearn.preprocessing import StandardScaler, RobustScaler

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# YOLO and OpenCV
from ultralytics import YOLO
import cv2

# Utilities
import warnings
import logging
import pickle
from scipy.signal import savgol_filter
from scipy.spatial.distance import euclidean
from collections import deque, defaultdict
import torch
import torch.nn as nn

os.environ['YOLO_VERBOSE'] = 'False'
logging.getLogger('ultralytics').setLevel(logging.WARNING)
warnings.filterwarnings('ignore')

print("All imports completed successfully")

All imports completed successfully


# Section 2: GPU Configuration
## Hardware Acceleration Setup

Configures GPU acceleration for TensorFlow and PyTorch environments. 

**Key Actions:**
- TensorFlow GPU memory configuration
- PyTorch CUDA device detection  
- YOLO device assignment (GPU/CPU)

In [2]:
print("\n" + "="*70)
print("GPU CONFIGURATION")
print("="*70)

# TensorFlow GPU setup
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(f"Physical GPUs: {len(gpus)}, Logical GPUs: {len(logical_gpus)}")
    except RuntimeError as e:
        print(f"GPU configuration error: {e}")
else:
    print("No GPU detected - using CPU")

# PyTorch GPU setup
print(f"\nPyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"PyTorch CUDA device: {torch.cuda.get_device_name(0)}")
    YOLO_DEVICE = 0
else:
    print("PyTorch CUDA not available - YOLO will use CPU")
    YOLO_DEVICE = 'cpu'

print("="*70)



GPU CONFIGURATION
Physical GPUs: 1, Logical GPUs: 1

PyTorch version: 1.12.1+cu113
CUDA available: True
PyTorch CUDA device: NVIDIA GeForce RTX 4070 SUPER


# Section 3: Configuration Parameters
## System Constants and Paths

Defines dataset paths, model parameters, feature dimensions, and YOLO keypoint mappings. Creates directories for output storage.

**Key Configurations:**
- Dataset paths (RWF-2000)
- Model architecture parameters
- Feature dimensions (10 total)
- YOLO keypoint indices
- Directory structure setup

In [3]:
# ============================================================================
# SECTION 3: CONFIGURATION PARAMETERS
# ============================================================================

# Dataset paths
FIGHT_PATH = r"D:\\RWF-2000 Dataset\\RWF2000\\RWF-2000\\train\\Fight"
NONFIGHT_PATH = r"D:\\RWF-2000 Dataset\\RWF2000\\RWF-2000\\train\\NonFight"

# Model paths
MODEL_PATH = "models/violence_detection_slr_aligned.keras"
FEATURES_PATH = "extracted_features_slr_aligned"
YOLO_MODEL = "yolov8l-pose.pt"

# Training parameters
GPU_BATCH_SIZE = 32
MIN_WINDOW_SIZE = 60  # SLR recommendation: 60-90 frames
MAX_WINDOW_SIZE = 90
CONFIDENCE_THRESHOLD = 0.25
TRACKING_THRESHOLD = 0.7

# Feature dimensions (SLR-aligned)
ANGLE_FEATURES = 4      # Elbow angles (L/R), shoulder, thorax-pelvis
DELTA_FEATURES = 4      # Temporal changes in angles
SPATIAL_FEATURES = 2    # Interpersonal distance, movement velocity
ROM_FEATURES = 2        # Range of motion metrics
FEATURE_DIM = 10        # Total: 4 angles + 4 deltas + 2 spatial

# Model architecture
LSTM_UNITS_1 = 128
LSTM_UNITS_2 = 64
LSTM_UNITS_3 = 32
DENSE_UNITS = 128
DROPOUT_RATE = 0.4
SPATIAL_DROPOUT = 0.3
LEARNING_RATE = 0.0003
EARLY_STOPPING_PATIENCE = 25

# Temporal parameters
SHORT_TERM_WINDOW = 30
TEMPORAL_BUFFER_SIZE = 10

# Feature names for interpretability
FEATURE_NAMES = [
    'Left Elbow Angle', 
    'Right Elbow Angle', 
    'Shoulder Angle', 
    'Thorax-Pelvis Rotation',
    'Delta Left Elbow', 
    'Delta Right Elbow', 
    'Delta Shoulder', 
    'Delta Thorax-Pelvis',
    'Interpersonal Distance',
    'Movement Velocity'
]

# YOLO keypoint indices
KEYPOINT_INDICES = {
    'nose': 0, 'left_eye': 1, 'right_eye': 2, 'left_ear': 3, 'right_ear': 4,
    'left_shoulder': 5, 'right_shoulder': 6, 'left_elbow': 7, 'right_elbow': 8,
    'left_wrist': 9, 'right_wrist': 10, 'left_hip': 11, 'right_hip': 12,
    'left_knee': 13, 'right_knee': 14, 'left_ankle': 15, 'right_ankle': 16
}

# Create directories
for directory in ['models', FEATURES_PATH, 'results', 'logs', 'visualizations']:
    os.makedirs(directory, exist_ok=True)

print("Configuration complete")
print(f"Feature dimension: {FEATURE_DIM}")
print(f"Features: {FEATURE_NAMES}")


Configuration complete
Feature dimension: 10
Features: ['Left Elbow Angle', 'Right Elbow Angle', 'Shoulder Angle', 'Thorax-Pelvis Rotation', 'Delta Left Elbow', 'Delta Right Elbow', 'Delta Shoulder', 'Delta Thorax-Pelvis', 'Interpersonal Distance', 'Movement Velocity']


# Section 4: Feature Extraction Class

Adds multi-person pose estimation with YOLOv8 and extracts biomechanical features for violence detection.

**Core Components:**
- YOLOv8 pose estimation with BoTSORT tracking
- Biomechanical angle computation (elbow, shoulder, thorax-pelvis)
- Temporal delta features and spatial metrics
- Violence potential scoring based on SLR heuristics

In [4]:
class SLRAlignedExtractor:
    """
    Feature extractor implementing SLR-aligned methodology:
    - Multi-person pose estimation with YOLOv8
    - Biomechanical angle computation
    - Temporal delta features
    - Spatial interaction metrics
    - BoTSORT-inspired tracking
    """
    
    def __init__(self):
        print(f"Initializing YOLO model on device: {YOLO_DEVICE}")
        self.pose_model = YOLO(YOLO_MODEL)
        self.previous_angles = None
        self.keypoint_history = []
        self.tracked_persons = {}
        self.next_person_id = 0
        self.rom_cycles = defaultdict(list)
        self.temporal_scores = deque(maxlen=SHORT_TERM_WINDOW)
    
    # ------------------------------------------------------------------------
    # Geometric Computation Methods
    # ------------------------------------------------------------------------
    
    def compute_angle(self, a, b, c):
        """
        Calculate angle between three points using law of cosines.
        
        Args:
            a, b, c: Points as (x, y) coordinates
            
        Returns:
            Angle in degrees at point b
        """
        try:
            if (a is None or b is None or c is None or 
                np.isnan(a).any() or np.isnan(b).any() or np.isnan(c).any()):
                return 0.0
            
            ba = np.array(a) - np.array(b)
            bc = np.array(c) - np.array(b)
            
            cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc) + 1e-8)
            cosine_angle = np.clip(cosine_angle, -1.0, 1.0)
            angle = np.degrees(np.arccos(cosine_angle))
            return angle
        except:
            return 0.0
    
    def compute_thorax_pelvis_rotation(self, keypoints):
        """
        Calculate thorax-pelvis rotation angle.
        Important for detecting twisting motions in violence.
        """
        try:
            if not (self.is_valid_keypoint(keypoints, 5) and 
                   self.is_valid_keypoint(keypoints, 6) and
                   self.is_valid_keypoint(keypoints, 11) and 
                   self.is_valid_keypoint(keypoints, 12)):
                return 0.0
            
            thorax_vec = np.array(keypoints[6]) - np.array(keypoints[5])
            pelvis_vec = np.array(keypoints[12]) - np.array(keypoints[11])
            
            cosine_angle = np.dot(thorax_vec, pelvis_vec) / \
                          (np.linalg.norm(thorax_vec) * np.linalg.norm(pelvis_vec) + 1e-8)
            cosine_angle = np.clip(cosine_angle, -1.0, 1.0)
            angle = np.degrees(np.arccos(cosine_angle))
            
            return angle
        except:
            return 0.0
    
    def compute_spatial_metrics(self, keypoints_list):
        """
        Calculate spatial interaction metrics between persons.
        Returns interpersonal distance and movement velocity.
        """
        if len(keypoints_list) < 2:
            return 0.0, 0.0
        
        # Interpersonal distance
        person1, person2 = keypoints_list[0], keypoints_list[1]
        shoulder_distances = []
        
        for i in [5, 6]:  # shoulder indices
            if (self.is_valid_keypoint(person1, i) and 
                self.is_valid_keypoint(person2, i)):
                dist = euclidean(person1[i], person2[i])
                shoulder_distances.append(dist)
        
        interpersonal_distance = np.mean(shoulder_distances) if shoulder_distances else 0.0
        
        # Movement velocity
        movement_velocity = 0.0
        if len(self.keypoint_history) >= 2:
            current_centroid = self.compute_centroid(keypoints_list[0])
            previous_centroid = self.compute_centroid(
                self.keypoint_history[-2][0] if len(self.keypoint_history) >= 2 
                else keypoints_list[0]
            )
            if current_centroid is not None and previous_centroid is not None:
                movement_velocity = euclidean(current_centroid, previous_centroid)
        
        return interpersonal_distance, movement_velocity
    
    def compute_centroid(self, keypoints):
        """Calculate centroid of valid keypoints."""
        valid_points = [kp for kp in keypoints 
                       if kp is not None and not np.isnan(kp).any()]
        return np.mean(valid_points, axis=0) if valid_points else None
    
    # ------------------------------------------------------------------------
    # Tracking Methods
    # ------------------------------------------------------------------------
    
    def bot_sort_tracking(self, current_detections, current_features):
        """
        Simplified BoTSORT-like multi-person tracking.
        Matches persons across frames using IoU and feature similarity.
        """
        matched_persons = {}
        
        for detection_idx, (bbox, features) in enumerate(zip(current_detections, current_features)):
            best_match_id = None
            best_similarity = -1
            
            for person_id, person_data in self.tracked_persons.items():
                iou = self.calculate_iou(bbox, person_data['bbox'])
                feature_sim = self.calculate_feature_similarity(
                    features, person_data['features']
                )
                
                similarity = 0.7 * iou + 0.3 * feature_sim
                
                if similarity > best_similarity and similarity > TRACKING_THRESHOLD:
                    best_similarity = similarity
                    best_match_id = person_id
            
            if best_match_id is not None:
                matched_persons[best_match_id] = {
                    'bbox': bbox,
                    'features': features,
                    'age': self.tracked_persons[best_match_id]['age'] + 1
                }
            else:
                new_id = self.next_person_id
                matched_persons[new_id] = {
                    'bbox': bbox,
                    'features': features,
                    'age': 1
                }
                self.next_person_id += 1
        
        self.tracked_persons = matched_persons
        return list(matched_persons.keys())
    
    def calculate_iou(self, bbox1, bbox2):
        """Calculate Intersection over Union for bounding boxes."""
        try:
            x1 = max(bbox1[0], bbox2[0])
            y1 = max(bbox1[1], bbox2[1])
            x2 = min(bbox1[2], bbox2[2])
            y2 = min(bbox1[3], bbox2[3])
            
            intersection = max(0, x2 - x1) * max(0, y2 - y1)
            area1 = (bbox1[2] - bbox1[0]) * (bbox1[3] - bbox1[1])
            area2 = (bbox2[2] - bbox2[0]) * (bbox2[3] - bbox2[1])
            
            union = area1 + area2 - intersection
            return intersection / union if union > 0 else 0.0
        except:
            return 0.0
    
    def calculate_feature_similarity(self, features1, features2):
        """Calculate normalized feature similarity."""
        try:
            if len(features1) != len(features2):
                return 0.0
            return 1.0 - np.linalg.norm(np.array(features1) - np.array(features2)) / len(features1)
        except:
            return 0.0
    
    # ------------------------------------------------------------------------
    # Feature Extraction Methods
    # ------------------------------------------------------------------------
    
    def extract_slr_features_enhanced(self, keypoints):
        """
        Extract 8 base biomechanical features:
        - 4 angle features (elbow L/R, shoulder, thorax-pelvis)
        - 4 temporal delta features
        """
        features = [0.0] * 8
        
        try:
            # Feature 1: Left elbow angle
            if (self.is_valid_keypoint(keypoints, 5) and 
                self.is_valid_keypoint(keypoints, 7) and 
                self.is_valid_keypoint(keypoints, 9)):
                features[0] = self.compute_angle(
                    keypoints[5], keypoints[7], keypoints[9]
                )
            
            # Feature 2: Right elbow angle
            if (self.is_valid_keypoint(keypoints, 6) and 
                self.is_valid_keypoint(keypoints, 8) and 
                self.is_valid_keypoint(keypoints, 10)):
                features[1] = self.compute_angle(
                    keypoints[6], keypoints[8], keypoints[10]
                )
            
            # Feature 3: Shoulder angle
            if (self.is_valid_keypoint(keypoints, 5) and 
                self.is_valid_keypoint(keypoints, 6)):
                shoulder_vec = np.array(keypoints[6]) - np.array(keypoints[5])
                horizontal_vec = np.array([1, 0])
                cosine_angle = np.dot(shoulder_vec, horizontal_vec) / \
                              (np.linalg.norm(shoulder_vec) + 1e-8)
                cosine_angle = np.clip(cosine_angle, -1.0, 1.0)
                features[2] = np.degrees(np.arccos(cosine_angle))
            
            # Feature 4: Thorax-pelvis rotation
            features[3] = self.compute_thorax_pelvis_rotation(keypoints)
            
            # Features 5-8: Temporal deltas
            if self.previous_angles is not None:
                for i in range(4):
                    features[4 + i] = features[i] - self.previous_angles[i]
            
            self.previous_angles = features[:4].copy()
            
            self.keypoint_history.append(keypoints)
            if len(self.keypoint_history) > 5:
                self.keypoint_history.pop(0)
            
        except Exception as e:
            print(f"Feature extraction error: {e}")
        
        return features
    
    def is_valid_keypoint(self, keypoints, index):
        """Check if a keypoint is valid and visible."""
        try:
            if (keypoints is None or index >= len(keypoints) or 
                keypoints[index] is None or np.isnan(keypoints[index]).any()):
                return False
            return True
        except:
            return False
    
    def extract_frame_features_enhanced(self, frame):
        """
        Main feature extraction pipeline for a single frame.
        Returns 10-dimensional feature vector.
        """
        try:
            # Run YOLO pose detection
            results = self.pose_model(
                frame, conf=CONFIDENCE_THRESHOLD, verbose=False, device=YOLO_DEVICE
            )
            
            if not results or len(results) == 0:
                return [0.0] * FEATURE_DIM
            
            result = results[0]
            
            if not hasattr(result, 'keypoints') or result.keypoints is None:
                return [0.0] * FEATURE_DIM
            
            try:
                keypoints_data = result.keypoints.data
                if keypoints_data is None or len(keypoints_data) == 0:
                    return [0.0] * FEATURE_DIM
            except:
                return [0.0] * FEATURE_DIM
            
            all_person_features = []
            all_keypoints = []
            bboxes = []
            
            # Process each detected person
            for person_idx in range(len(keypoints_data)):
                try:
                    person_keypoints = keypoints_data[person_idx]
                    
                    if hasattr(person_keypoints, 'cpu'):
                        keypoints_np = person_keypoints.cpu().numpy()
                    else:
                        keypoints_np = person_keypoints
                    
                    if hasattr(result.keypoints, 'conf'):
                        confidences = result.keypoints.conf[person_idx].cpu().numpy() \
                                     if hasattr(result.keypoints.conf[person_idx], 'cpu') \
                                     else result.keypoints.conf[person_idx]
                    else:
                        confidences = np.ones(len(keypoints_np))
                    
                    if (hasattr(result, 'boxes') and result.boxes is not None and 
                        len(result.boxes) > person_idx):
                        bbox = result.boxes.xyxy[person_idx].cpu().numpy() \
                              if hasattr(result.boxes.xyxy[person_idx], 'cpu') \
                              else result.boxes.xyxy[person_idx]
                    else:
                        bbox = [0, 0, 1, 1]
                    
                    # Validate keypoints
                    validated_keypoints = []
                    for i in range(len(keypoints_np)):
                        kp = keypoints_np[i]
                        conf = confidences[i] if i < len(confidences) else 1.0
                        
                        if (kp is not None and len(kp) >= 2 and 
                            not np.isnan(kp).any() and conf > CONFIDENCE_THRESHOLD):
                            validated_keypoints.append((float(kp[0]), float(kp[1])))
                        else:
                            validated_keypoints.append(None)
                    
                    features = self.extract_slr_features_enhanced(validated_keypoints)
                    
                    if len(features) < 8:
                        features.extend([0.0] * (8 - len(features)))
                    elif len(features) > 8:
                        features = features[:8]
                    
                    all_person_features.append(features)
                    all_keypoints.append(validated_keypoints)
                    bboxes.append(bbox)
                    
                except Exception as e:
                    continue
            
            if not all_person_features:
                return [0.0] * FEATURE_DIM
            
            # Multi-person tracking
            if len(all_person_features) > 1:
                try:
                    person_ids = self.bot_sort_tracking(bboxes, all_person_features)
                except:
                    person_ids = list(range(len(all_person_features)))
            else:
                person_ids = [0]
            
            # Spatial metrics
            try:
                if len(all_keypoints) >= 2:
                    spatial_metrics = self.compute_spatial_metrics(all_keypoints)
                else:
                    spatial_metrics = (0.0, 0.0)
            except:
                spatial_metrics = (0.0, 0.0)
            
            person_violence_scores = []
            final_features_list = []
            
            for i, (features, person_id) in enumerate(zip(all_person_features, person_ids)):
                try:
                    enhanced_features = features + list(spatial_metrics)
                    
                    if len(enhanced_features) < FEATURE_DIM:
                        enhanced_features.extend([0.0] * (FEATURE_DIM - len(enhanced_features)))
                    
                    violence_score = self.calculate_violence_potential_slr(
                        enhanced_features, person_id
                    )
                    person_violence_scores.append(violence_score)
                    final_features_list.append(enhanced_features)
                except:
                    continue
            
            if final_features_list:
                if len(final_features_list) == 1:
                    return final_features_list[0]
                else:
                    try:
                        max_violence_idx = np.argmax(person_violence_scores)
                        return final_features_list[max_violence_idx]
                    except:
                        return final_features_list[0]
            else:
                return [0.0] * FEATURE_DIM
                
        except Exception as e:
            return [0.0] * FEATURE_DIM
    
    def calculate_violence_potential_slr(self, features, person_id):
        """
        Calculate violence potential score using SLR-inspired heuristics.
        """
        base_score = 0.0
        
        try:
            angles = features[:4]
            deltas = features[4:8]
            spatial = features[8:] if len(features) > 8 else [0.0, 0.0]
            
            # Extreme angle detection
            extreme_angles = sum(1 for angle in angles if angle > 120 or angle < 30)
            base_score += extreme_angles * 0.25
            
            # Rapid movement detection
            rapid_movements = sum(1 for delta in deltas if abs(delta) > 5)
            base_score += rapid_movements * 0.35
            
            # Thorax-pelvis rotation
            if angles[3] > 45:
                base_score += 0.3
            
            # Spatial aggression indicators
            if spatial[0] > 0 and spatial[0] < 0.1:
                base_score += 0.2
            if spatial[1] > 0.05:
                base_score += 0.15
            
        except:
            pass
        
        # Temporal smoothing
        self.temporal_scores.append(base_score)
        if len(self.temporal_scores) >= SHORT_TERM_WINDOW:
            smoothed_score = max(self.temporal_scores)
        else:
            smoothed_score = base_score
        
        return min(smoothed_score, 1.0)
    
    # ------------------------------------------------------------------------
    # Video Processing Methods
    # ------------------------------------------------------------------------
    
    def process_video_enhanced(self, video_path, max_frames=None):
        """
        Process entire video and extract feature windows.
        """
        print(f"Processing: {os.path.basename(video_path)}")
        
        try:
            cap = cv2.VideoCapture(video_path)
            if not cap.isOpened():
                print(f"  Cannot open video: {video_path}")
                return []
            
            original_fps = cap.get(cv2.CAP_PROP_FPS)
            total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            
            if total_frames == 0 or original_fps == 0:
                print(f"  Invalid video: {video_path}")
                cap.release()
                return []
            
            # Adaptive window sizing
            if original_fps < 20:
                window_size = 60
                stride = 30
            else:
                window_size = 75
                stride = 38
            
            print(f"  FPS={original_fps:.1f}, Window={window_size}, Stride={stride}")
            
            video_features = []
            frame_count = 0
            
            # Reset tracking
            self.previous_angles = None
            self.keypoint_history = []
            self.tracked_persons = {}
            self.next_person_id = 0
            self.rom_cycles.clear()
            self.temporal_scores.clear()
            
            while True:
                ret, frame = cap.read()
                if not ret or (max_frames and frame_count >= max_frames):
                    break
                
                features = self.extract_frame_features_enhanced(frame)
                video_features.append(features)
                frame_count += 1
                
                if frame_count % 50 == 0:
                    print(f"    Processed {frame_count}/{total_frames} frames")
            
            cap.release()
            
            # Feature smoothing
            if len(video_features) >= 11:
                try:
                    video_features = self.smooth_features(video_features)
                except:
                    pass
            
            # Create windows
            video_windows = self.create_windows(video_features, window_size, stride)
            print(f"  Extracted {len(video_windows)} windows from {len(video_features)} frames")
            return video_windows
            
        except Exception as e:
            print(f"  Error processing {video_path}: {e}")
            try:
                cap.release()
            except:
                pass
            return []
    
    def smooth_features(self, features):
        """Apply Savitzky-Golay filter for temporal smoothing."""
        try:
            features_array = np.array(features)
            smoothed = np.zeros_like(features_array)
            
            for i in range(FEATURE_DIM):
                try:
                    smoothed[:, i] = savgol_filter(
                        features_array[:, i], window_length=11, polyorder=3
                    )
                except:
                    smoothed[:, i] = features_array[:, i]
            
            return smoothed.tolist()
        except:
            return features
    
    def create_windows(self, video_features, window_size, stride):
        """Create sliding temporal windows."""
        try:
            if len(video_features) < window_size:
                if len(video_features) >= 30:
                    window = video_features
                    if len(window) < window_size:
                        padding = [[0.0] * FEATURE_DIM] * (window_size - len(window))
                        window.extend(padding)
                    return [window]
                else:
                    return []
            
            windows = []
            for start_idx in range(0, len(video_features) - window_size + 1, stride):
                window = video_features[start_idx:start_idx + window_size]
                windows.append(window)
            
            return windows
        except:
            return []

print("Feature extractor initialized successfully")

Feature extractor initialized successfully


# Section 5: Training System
## Complete Violence Detection Pipeline

Create end-to-end training with feature extraction, dataset preparation, model architecture, and evaluation.

**Core Pipeline:**
- Feature extraction from RWF-2000 videos
- Dataset balancing with augmentation
- BiLSTM model with attention mechanism
- Comprehensive evaluation metrics
- Model saving and visualization

In [5]:
# ============================================================================
# SECTION 5: TRAINING SYSTEM
# ============================================================================

class SLRAlignedViolenceDetection:
    """
    Complete training pipeline for violence detection:
    - Feature extraction from videos
    - Dataset preparation with augmentation
    - Model training with BiLSTM + attention
    - Evaluation with comprehensive metrics
    """
    
    def __init__(self):
        self.model = None
        self.history = None
        self.extractor = SLRAlignedExtractor()
        self.results_dir = "slr_aligned_results"
        self.visualization_dir = "visualizations"
        self.scaler_path = os.path.join(FEATURES_PATH, "slr_aligned_scaler.pkl")
        self.scaler = None
        self.class_weights = None

        for directory in [self.results_dir, self.visualization_dir, FEATURES_PATH]:
            os.makedirs(directory, exist_ok=True)
    
    # ------------------------------------------------------------------------
    # Feature Extraction
    # ------------------------------------------------------------------------
    
    def extract_features(self, fight_path, nonfight_path, 
                        max_videos_per_class=None, max_frames_per_video=None):
        """
        Extract features from fight and non-fight videos.
        """
        print(f"\n{'='*70}")
        print("FEATURE EXTRACTION")
        print("="*70)

        # Process fight videos
        fight_features = []
        fight_videos = [f for f in os.listdir(fight_path) 
                       if f.lower().endswith(('.mp4', '.avi', '.mov', '.mkv'))]
        if max_videos_per_class:
            fight_videos = fight_videos[:max_videos_per_class]

        print(f"Processing {len(fight_videos)} FIGHT videos...")
        for i, video_file in enumerate(fight_videos):
            print(f"  [{i+1}/{len(fight_videos)}] {video_file}")
            video_path = os.path.join(fight_path, video_file)
            windows = self.extractor.process_video_enhanced(
                video_path, max_frames=max_frames_per_video
            )
            fight_features.extend(windows)

        # Process non-fight videos
        nonfight_features = []
        nonfight_videos = [f for f in os.listdir(nonfight_path) 
                          if f.lower().endswith(('.mp4', '.avi', '.mov', '.mkv'))]
        if max_videos_per_class:
            nonfight_videos = nonfight_videos[:max_videos_per_class]

        print(f"Processing {len(nonfight_videos)} NON-FIGHT videos...")
        for i, video_file in enumerate(nonfight_videos):
            print(f"  [{i+1}/{len(nonfight_videos)}] {video_file}")
            video_path = os.path.join(nonfight_path, video_file)
            windows = self.extractor.process_video_enhanced(
                video_path, max_frames=max_frames_per_video
            )
            nonfight_features.extend(windows)

        # Combine features and labels
        all_features = fight_features + nonfight_features
        all_labels = [1] * len(fight_features) + [0] * len(nonfight_features)

        print(f"\nFeature Summary:")
        print(f"  Fight samples: {len(fight_features)}")
        print(f"  Non-Fight samples: {len(nonfight_features)}")
        print(f"  Total samples: {len(all_features)}")
        print(f"  Feature dimension: {FEATURE_DIM}")

        if len(all_features) == 0:
            print("ERROR: No features extracted!")
            return None, None

        # Save features
        np.save(
            os.path.join(FEATURES_PATH, 'slr_aligned_features.npy'), 
            np.array(all_features, dtype=np.float32)
        )
        np.save(
            os.path.join(FEATURES_PATH, 'slr_aligned_labels.npy'), 
            np.array(all_labels, dtype=np.int32)
        )

        print(f"Features saved to {FEATURES_PATH}/")
        return all_features, all_labels
    
    # ------------------------------------------------------------------------
    # Dataset Preparation
    # ------------------------------------------------------------------------
    
    def prepare_dataset(self, test_size=0.2, validation_size=0.15, 
                       balance=True, random_state=42):
        """
        Prepare train/validation/test splits with normalization and balancing.
        """
        print(f"\n{'='*70}")
        print("DATASET PREPARATION")
        print("="*70)

        features_path = os.path.join(FEATURES_PATH, 'slr_aligned_features.npy')
        labels_path = os.path.join(FEATURES_PATH, 'slr_aligned_labels.npy')

        if not (os.path.exists(features_path) and os.path.exists(labels_path)):
            print("ERROR: No features found! Run extraction first.")
            return None, None, None, None, None, None

        X = np.load(features_path)
        y = np.load(labels_path)

        print(f"Loaded {len(X)} samples")
        print(f"Class distribution - Class 0: {np.sum(y == 0)}, Class 1: {np.sum(y == 1)}")

        # Train/test split
        X_temp, X_test, y_temp, y_test = train_test_split(
            X, y, test_size=test_size, random_state=random_state, 
            stratify=y, shuffle=True
        )

        # Train/validation split
        val_size_adjusted = validation_size / (1 - test_size)
        X_train, X_val, y_train, y_val = train_test_split(
            X_temp, y_temp, test_size=val_size_adjusted, random_state=random_state, 
            stratify=y_temp, shuffle=True
        )

        print(f"\nSplit sizes:")
        print(f"  Train: {len(y_train)} (0:{np.sum(y_train==0)}, 1:{np.sum(y_train==1)})")
        print(f"  Val:   {len(y_val)} (0:{np.sum(y_val==0)}, 1:{np.sum(y_val==1)})")
        print(f"  Test:  {len(y_test)} (0:{np.sum(y_test==0)}, 1:{np.sum(y_test==1)})")

        # Normalize features
        self.scaler = RobustScaler()
        X_train_flat = X_train.reshape(-1, FEATURE_DIM)
        X_train_flat_norm = self.scaler.fit_transform(X_train_flat)
        X_train_norm = X_train_flat_norm.reshape(X_train.shape)

        X_val_flat = X_val.reshape(-1, FEATURE_DIM)
        X_val_flat_norm = self.scaler.transform(X_val_flat)
        X_val_norm = X_val_flat_norm.reshape(X_val.shape)

        X_test_flat = X_test.reshape(-1, FEATURE_DIM)
        X_test_flat_norm = self.scaler.transform(X_test_flat)
        X_test_norm = X_test_flat_norm.reshape(X_test.shape)

        # Save scaler
        with open(self.scaler_path, "wb") as f:
            pickle.dump(self.scaler, f)
        print(f"Scaler saved to {self.scaler_path}")

        # Balance dataset with augmentation
        if balance:
            X_train_norm, y_train = self._balance_with_augmentation(X_train_norm, y_train)

        # Calculate class weights
        classes = np.unique(y_train)
        weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
        self.class_weights = {int(c): float(w) for c, w in zip(classes, weights)}
        print(f"Class weights: {self.class_weights}")

        # Save processed data
        np.save(os.path.join(FEATURES_PATH, 'X_train_slr_aligned.npy'), X_train_norm)
        np.save(os.path.join(FEATURES_PATH, 'X_val_slr_aligned.npy'), X_val_norm)
        np.save(os.path.join(FEATURES_PATH, 'X_test_slr_aligned.npy'), X_test_norm)
        np.save(os.path.join(FEATURES_PATH, 'y_train_slr_aligned.npy'), y_train)
        np.save(os.path.join(FEATURES_PATH, 'y_val_slr_aligned.npy'), y_val)
        np.save(os.path.join(FEATURES_PATH, 'y_test_slr_aligned.npy'), y_test)

        print(f"\nFinal shapes - Train: {X_train_norm.shape}, Val: {X_val_norm.shape}, Test: {X_test_norm.shape}")
        return X_train_norm, X_val_norm, X_test_norm, y_train, y_val, y_test
    
    def _balance_with_augmentation(self, X, y):
        """Balance dataset using data augmentation techniques."""
        pos_idx = np.where(y == 1)[0]
        neg_idx = np.where(y == 0)[0]
        n_pos, n_neg = len(pos_idx), len(neg_idx)
        
        print(f"Before balancing - Pos: {n_pos}, Neg: {n_neg}")
        
        if n_pos == 0 or n_neg == 0:
            return X, y
        
        # Determine minority class
        if n_pos < n_neg:
            minority_idx = pos_idx
            samples_needed = n_neg - n_pos
        else:
            minority_idx = neg_idx
            samples_needed = n_pos - n_neg
        
        augmented_X = []
        augmented_y = []
        
        # Generate augmented samples
        for _ in range(samples_needed):
            idx = np.random.choice(minority_idx)
            sample = X[idx].copy()
            
            aug_type = np.random.randint(0, 4)
            
            if aug_type == 0:  # Add noise
                noise = np.random.normal(0, 0.03, sample.shape)
                sample = sample + noise
            elif aug_type == 1:  # Time shift
                shift = np.random.randint(-3, 4)
                sample = np.roll(sample, shift, axis=0)
            elif aug_type == 2:  # Scaling
                scale = np.random.uniform(0.95, 1.05)
                sample = sample * scale
            else:  # Time stretching
                indices = np.arange(len(sample))
                stretched = np.linspace(
                    0, len(sample)-1, int(len(sample)*np.random.uniform(0.95, 1.05))
                )
                for feat_idx in range(sample.shape[1]):
                    sample[:, feat_idx] = np.interp(
                        indices, stretched, 
                        np.interp(stretched, indices, sample[:, feat_idx])
                    )
            
            augmented_X.append(sample)
            augmented_y.append(y[idx])
        
        # Combine and shuffle
        X_balanced = np.concatenate([X, np.array(augmented_X)], axis=0)
        y_balanced = np.concatenate([y, np.array(augmented_y)], axis=0)
        
        shuffle_idx = np.random.permutation(len(X_balanced))
        X_balanced = X_balanced[shuffle_idx]
        y_balanced = y_balanced[shuffle_idx]
        
        print(f"After balancing - Total: {len(X_balanced)}, Pos: {np.sum(y_balanced==1)}, Neg: {np.sum(y_balanced==0)}")
        return X_balanced, y_balanced
    
    # ------------------------------------------------------------------------
    # Model Architecture
    # ------------------------------------------------------------------------
    
    def build_improved_model(self, input_shape, initial_bias=None):
        """
        Build BiLSTM model with attention mechanism.
        Simplified architecture with strong regularization.
        """
        output_bias = tf.keras.initializers.Constant(initial_bias) \
                     if initial_bias is not None else 'zeros'

        inputs = layers.Input(shape=input_shape, name='input_layer')
        
        # First BiLSTM layer
        x = layers.Bidirectional(
            layers.LSTM(64, return_sequences=True, dropout=0.3, recurrent_dropout=0.2)
        )(inputs)
        x = layers.BatchNormalization()(x)
        
        # Second BiLSTM layer
        x = layers.Bidirectional(
            layers.LSTM(32, return_sequences=False, dropout=0.3, recurrent_dropout=0.2)
        )(x)
        x = layers.BatchNormalization()(x)
        
        x = layers.Dropout(0.5)(x)
        
        # Dense layers
        x = layers.Dense(64, activation='relu', 
                        kernel_regularizer=regularizers.l2(0.01))(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.4)(x)
        
        x = layers.Dense(32, activation='relu', 
                        kernel_regularizer=regularizers.l2(0.01))(x)
        x = layers.BatchNormalization()(x)
        
        # Output layer
        outputs = layers.Dense(1, activation='sigmoid', 
                              bias_initializer=output_bias, name='output')(x)
        
        model = models.Model(inputs=inputs, outputs=outputs, 
                           name='SLRViolenceDetector')
        
        # Compile model
        model.compile(
            optimizer=optimizers.Adam(learning_rate=0.0001, clipnorm=1.0),
            loss='binary_crossentropy',
            metrics=[
                'accuracy',
                tf.keras.metrics.AUC(name='auc', curve='ROC'),
                tf.keras.metrics.Precision(name='precision'),
                tf.keras.metrics.Recall(name='recall'),
                tf.keras.metrics.AUC(name='pr_auc', curve='PR')
            ]
        )

        print("\nMODEL ARCHITECTURE")
        print("="*50)
        print(f"Input shape: {input_shape}")
        print(f"Feature dimension: {FEATURE_DIM}")
        model.summary()
        
        self.model = model
        return model
    
    # ------------------------------------------------------------------------
    # Training
    # ------------------------------------------------------------------------
    
    def train_model(self, X_train, y_train, X_val, y_val, epochs=150):
        """
        Train the violence detection model.
        """
        print(f"\n{'='*70}")
        print("MODEL TRAINING")
        print("="*70)
        
        print(f"Train: Pos={np.sum(y_train==1)}, Neg={np.sum(y_train==0)}")
        print(f"Val:   Pos={np.sum(y_val==1)}, Neg={np.sum(y_val==0)}")
    
        # Calculate initial bias
        pos = float(np.sum(y_train))
        neg = float(len(y_train) - pos)
        initial_bias = np.log((pos + 1e-7) / (neg + 1e-7)) \
                      if pos > 0 and neg > 0 else None
        print(f"Initial bias: {initial_bias:.4f}" if initial_bias else "No bias")
    
        # Build model
        input_shape = (X_train.shape[1], X_train.shape[2])
        self.build_improved_model(input_shape, initial_bias=initial_bias)
    
        # Callbacks
        callbacks = [
            tf.keras.callbacks.EarlyStopping(
                monitor='val_auc',
                patience=15,
                restore_best_weights=True,
                mode='max',
                min_delta=0.001,
                verbose=1
            ),
            tf.keras.callbacks.ModelCheckpoint(
                filepath=MODEL_PATH,
                monitor='val_auc',
                save_best_only=True,
                mode='max',
                verbose=1
            ),
            tf.keras.callbacks.ReduceLROnPlateau(
                monitor='val_loss',
                factor=0.5,
                patience=5,
                min_lr=1e-6,
                verbose=1
            ),
            tf.keras.callbacks.CSVLogger(
                os.path.join('logs', 'training_log.csv')
            )
        ]
    
        try:
            batch_size = min(16, GPU_BATCH_SIZE)
            print(f"Batch size: {batch_size}")
            print("Starting training...")
            
            self.history = self.model.fit(
                X_train, y_train.astype(np.float32),
                validation_data=(X_val, y_val.astype(np.float32)),
                epochs=epochs,
                batch_size=batch_size,
                class_weight=None,
                callbacks=callbacks,
                verbose=2,
                shuffle=True
            )
    
            self.model.save(MODEL_PATH)
            print(f"\nModel saved to: {MODEL_PATH}")
            
            self.plot_training_history()
            
            return self.history
            
        except Exception as e:
            print(f"Training error: {e}")
            return None
    
    def plot_training_history(self):
        """Plot training metrics over epochs."""
        try:
            if self.history is None:
                return
                
            fig, axes = plt.subplots(2, 2, figsize=(15, 10))
            
            # Accuracy
            axes[0, 0].plot(self.history.history['accuracy'], label='Train')
            axes[0, 0].plot(self.history.history['val_accuracy'], label='Validation')
            axes[0, 0].set_title('Model Accuracy')
            axes[0, 0].set_xlabel('Epoch')
            axes[0, 0].set_ylabel('Accuracy')
            axes[0, 0].legend()
            axes[0, 0].grid(True)
            
            # Loss
            axes[0, 1].plot(self.history.history['loss'], label='Train')
            axes[0, 1].plot(self.history.history['val_loss'], label='Validation')
            axes[0, 1].set_title('Model Loss')
            axes[0, 1].set_xlabel('Epoch')
            axes[0, 1].set_ylabel('Loss')
            axes[0, 1].legend()
            axes[0, 1].grid(True)
            
            # AUC
            axes[1, 0].plot(self.history.history['auc'], label='Train')
            axes[1, 0].plot(self.history.history['val_auc'], label='Validation')
            axes[1, 0].set_title('Model AUC')
            axes[1, 0].set_xlabel('Epoch')
            axes[1, 0].set_ylabel('AUC')
            axes[1, 0].legend()
            axes[1, 0].grid(True)
            
            # PR AUC
            if 'pr_auc' in self.history.history:
                axes[1, 1].plot(self.history.history['pr_auc'], label='Train')
                axes[1, 1].plot(self.history.history['val_pr_auc'], label='Validation')
                axes[1, 1].set_title('Precision-Recall AUC')
                axes[1, 1].set_xlabel('Epoch')
                axes[1, 1].set_ylabel('PR AUC')
                axes[1, 1].legend()
                axes[1, 1].grid(True)
            
            plt.tight_layout()
            plt.savefig(
                os.path.join(self.visualization_dir, 'training_history.png'), 
                dpi=300, bbox_inches='tight'
            )
            plt.close()
            
            print(f"Training history plot saved to {self.visualization_dir}/")
            
        except Exception as e:
            print(f"Could not plot training history: {e}")
    
    # ------------------------------------------------------------------------
    # Model Loading & Evaluation
    # ------------------------------------------------------------------------
    
    def load_model(self):
        """Load trained model and scaler from disk."""
        if os.path.exists(MODEL_PATH):
            try:
                self.model = tf.keras.models.load_model(MODEL_PATH)
                print(f"Model loaded from: {MODEL_PATH}")
                
                try:
                    with open(self.scaler_path, "rb") as f:
                        self.scaler = pickle.load(f)
                    print(f"Scaler loaded from: {self.scaler_path}")
                except Exception as e:
                    print(f"Warning: could not load scaler: {e}")
                
                return True
            except Exception as e:
                print(f"Error loading model: {e}")
                return False
        else:
            print("No trained model found!")
            return False
    
    def evaluate_model(self, X_test, y_test):
        """
        Comprehensive model evaluation with multiple metrics.
        """
        print(f"\n{'='*70}")
        print("MODEL EVALUATION")
        print("="*70)
        
        print(f"Test set size: {len(y_test)} samples")
        print(f"Class distribution - 0: {np.sum(y_test == 0)}, 1: {np.sum(y_test == 1)}")
        
        if self.model is None:
            print("No model loaded!")
            return None

        print("\nRunning predictions...")
        try:
            probs = self.model.predict(
                X_test, batch_size=GPU_BATCH_SIZE, verbose=1
            ).flatten()
            
            # Find optimal threshold
            fpr, tpr, thresholds = roc_curve(y_test, probs)
            optimal_idx = np.argmax(tpr - fpr)
            optimal_threshold = thresholds[optimal_idx]
            print(f"Optimal threshold: {optimal_threshold:.3f}")
            
            preds = (probs > optimal_threshold).astype(int)

            # Calculate metrics
            accuracy = accuracy_score(y_test, preds)
            precision = precision_score(y_test, preds, zero_division=0)
            recall = recall_score(y_test, preds, zero_division=0)
            f1 = f1_score(y_test, preds, zero_division=0)
            
            if len(np.unique(y_test)) > 1:
                roc_auc = roc_auc_score(y_test, probs)
                avg_precision = average_precision_score(y_test, probs)
            else:
                roc_auc = 0.5
                avg_precision = 0.5
                
            mcc = matthews_corrcoef(y_test, preds) \
                 if len(np.unique(preds)) > 1 else 0.0
            
            # Confusion matrix
            cm = confusion_matrix(y_test, preds)
            tn, fp, fn, tp = cm.ravel()
            specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
            balanced_accuracy = (recall + specificity) / 2

            # Print results
            print("\n" + "="*70)
            print("PERFORMANCE METRICS")
            print("="*70)
            print(f"Accuracy:             {accuracy:.4f} ({accuracy*100:.2f}%)")
            print(f"Balanced Accuracy:    {balanced_accuracy:.4f} ({balanced_accuracy*100:.2f}%)")
            print(f"Precision:            {precision:.4f} ({precision*100:.2f}%)")
            print(f"Recall (Sensitivity): {recall:.4f} ({recall*100:.2f}%)")
            print(f"Specificity:          {specificity:.4f} ({specificity*100:.2f}%)")
            print(f"F1-Score:             {f1:.4f}")
            print(f"ROC AUC:              {roc_auc:.4f}")
            print(f"PR AUC:               {avg_precision:.4f}")
            print(f"MCC:                  {mcc:.4f}")
            print("="*70)

            # Performance assessment
            if accuracy >= 0.85 and f1 >= 0.85:
                print("\nEXCELLENT: Target achieved (85%+)")
            elif accuracy >= 0.80 and f1 >= 0.80:
                print("\nGOOD: Strong performance (80%+)")
            elif accuracy >= 0.75:
                print("\nACCEPTABLE: Moderate performance (75%+)")
            else:
                print("\nNEEDS IMPROVEMENT: Below target (<75%)")

            # Plot confusion matrix
            self.plot_confusion_matrix(cm, accuracy)

            # Save results
            results = {
                'accuracy': float(accuracy),
                'balanced_accuracy': float(balanced_accuracy),
                'precision': float(precision),
                'recall': float(recall),
                'specificity': float(specificity),
                'f1_score': float(f1),
                'roc_auc': float(roc_auc),
                'pr_auc': float(avg_precision),
                'mcc': float(mcc),
                'optimal_threshold': float(optimal_threshold),
                'confusion_matrix': cm.tolist()
            }
            
            import json
            with open(os.path.join(self.results_dir, 'evaluation_results.json'), 'w') as f:
                json.dump(results, f, indent=4)
            print(f"\nResults saved to {self.results_dir}/")

            return results
            
        except Exception as e:
            print(f"Evaluation error: {e}")
            return None
    
    def plot_confusion_matrix(self, cm, accuracy):
        """Visualize confusion matrix."""
        try:
            plt.figure(figsize=(8, 6))
            sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                       xticklabels=['Non-Fight', 'Fight'],
                       yticklabels=['Non-Fight', 'Fight'])
            plt.title(f'Confusion Matrix (Accuracy: {accuracy:.2%})')
            plt.xlabel('Predicted Label')
            plt.ylabel('True Label')
            plt.savefig(
                os.path.join(self.visualization_dir, 'confusion_matrix.png'), 
                dpi=300, bbox_inches='tight'
            )
            plt.close()
            print(f"Confusion matrix saved to {self.visualization_dir}/")
        except Exception as e:
            print(f"Could not plot confusion matrix: {e}")


# Section 6: Interactive Menu System
## User Interface for Pipeline Control

**Menu Options:**
1. Extract Features from Videos
2. Train Model  
3. Evaluate Model
4. Run Full Pipeline
5. Exit

**Features:**
- Parameter configuration via user input
- Progress tracking for full pipeline

In [6]:
def run_interactive_menu():
    """
    Interactive menu for running different pipeline stages.
    """
    system = SLRAlignedViolenceDetection()
    
    while True:
        print(f"\n{'='*70}")
        print("SLR-ALIGNED VIOLENCE DETECTION SYSTEM")
        print("="*70)
        print("1. Extract Features from Videos")
        print("2. Train Model")
        print("3. Evaluate Model")
        print("4. Run Full Pipeline")
        print("5. Exit")
        
        choice = input("\nSelect option (1-5): ").strip()
        
        if choice == "1":
            # Extract features
            try:
                max_videos = input("Max videos per class (press Enter for all): ").strip()
                max_videos = int(max_videos) if max_videos.isdigit() else None
                
                max_frames = input("Max frames per video (press Enter for all): ").strip()
                max_frames = int(max_frames) if max_frames.isdigit() else None
                
                features, labels = system.extract_features(
                    FIGHT_PATH, NONFIGHT_PATH, 
                    max_videos_per_class=max_videos, 
                    max_frames_per_video=max_frames
                )
                
                if features is not None:
                    print("\nFeature extraction completed successfully")
            except Exception as e:
                print(f"Error: {e}")
        
        elif choice == "2":
            # Train model
            try:
                data = system.prepare_dataset(test_size=0.2, validation_size=0.15)
                
                if data[0] is None:
                    print("No data available!")
                    continue
                    
                X_train, X_val, X_test, y_train, y_val, y_test = data
                
                epochs = input("Number of epochs (press Enter for 150): ").strip()
                epochs = int(epochs) if epochs.isdigit() else 150
                
                history = system.train_model(X_train, y_train, X_val, y_val, epochs=epochs)
                
                if history is not None:
                    print("\nTraining completed successfully")
            except Exception as e:
                print(f"Error: {e}")
        
        elif choice == "3":
            # Evaluate model
            try:
                if not system.load_model():
                    continue
                    
                X_test_path = os.path.join(FEATURES_PATH, 'X_test_slr_aligned.npy')
                y_test_path = os.path.join(FEATURES_PATH, 'y_test_slr_aligned.npy')
                
                if not (os.path.exists(X_test_path) and os.path.exists(y_test_path)):
                    print("No test data found!")
                    continue
                    
                X_test = np.load(X_test_path)
                y_test = np.load(y_test_path)
                
                metrics = system.evaluate_model(X_test, y_test)
                
                if metrics:
                    print("\nEvaluation completed successfully")
            except Exception as e:
                print(f"Error: {e}")
        
        elif choice == "4":
            # Full pipeline
            try:
                print("\n[1/4] Feature Extraction")
                features, labels = system.extract_features(
                    FIGHT_PATH, NONFIGHT_PATH, 
                    max_videos_per_class=None,
                    max_frames_per_video=150
                )
                
                if features is None:
                    continue
                
                print("\n[2/4] Dataset Preparation")
                data = system.prepare_dataset(test_size=0.2, validation_size=0.15)
                if data[0] is None:
                    continue
                    
                X_train, X_val, X_test, y_train, y_val, y_test = data
                
                print("\n[3/4] Model Training")
                history = system.train_model(X_train, y_train, X_val, y_val, epochs=150)
                
                if history is None:
                    continue
                
                print("\n[4/4] Model Evaluation")
                metrics = system.evaluate_model(X_test, y_test)
                
                if metrics:
                    print("\n" + "="*70)
                    print("PIPELINE COMPLETED")
                    print("="*70)
                    print(f"Final Accuracy: {metrics['accuracy']*100:.2f}%")
                    print(f"Final F1-Score: {metrics['f1_score']:.4f}")
            except Exception as e:
                print(f"Error: {e}")
        
        elif choice == "5":
            print("\nExiting...")
            break
        
        else:
            print("Invalid choice! Please select 1-5")
        
        input("\nPress Enter to continue...")

# Section 7: Main Execution
## System Launch

Entry point for the violence detection system. Defines all components and starts the interface.


In [ ]:
if __name__ == "__main__":
    """
    Main execution block.
    Run this cell to start the interactive menu.
    """
    print("\n" + "="*70)
    print("INITIALIZING SLR-ALIGNED VIOLENCE DETECTION SYSTEM")
    print("="*70)
    print("\nSystem components:")
    print("- YOLOv8 pose estimation")
    print("- Multi-person tracking (BoTSORT-inspired)")
    print("- BiLSTM with attention mechanism")
    print(f"- {FEATURE_DIM}-dimensional feature space")
    print(f"- Target: 80-90% accuracy")
    print("\nStarting interactive menu...")
    
    run_interactive_menu()